# State (상태)

    유한상태기계처럼 조건마다 동일한 요청이 다른 방식으로 처리

## 정의

내부 상태(조건)이 변경될 때 마다 객체가 동작하는 방식을 변경할 수 있도록 하는 패턴.

휴대폰을 보자. 화면이꺼진상태(상태)에서 전원 버튼을 누르면(메서드) 화면이 켜진다. 그러나 화면이켜진상태에서 화면 버튼을 누르면 화면이 꺼진다. 상태에 따라 다르게 동작한다는 뜻은 이러한 방식(유한상태기계)을 의미한다.

phone.click_power_button이 phone.state.click_power_button을 호출한다. state에는 화면꺼진상태, 화면켜진상태가 있을 수 있다. phone.state가 화면꺼진상태였다면, 결국 phone.화면꺼진상태.click_power_button이 호출되는 것이다. 화면꺼진상태.click_power_button은 호출될 경우 휴대폰의 화면을 켠 다음 phone.state를 화면꺼진상태로 변경한다.

전략 패턴에 다음 알고리즘(전략 객체의 동작 수행 과정에 다른 전략 객체로의 교환 과정을 포함)을 포함하도록 구성된 패턴이라고 볼 수 있다.

## 구현

In [18]:
from __future__ import annotations


class AudioPlayer:

    def __init__(self):
        self.state: State = LockedState(self)

    def start_playback(self):
        print("AudioPlayer: play song")

    def stop_playback(self):
        print("AudioPlayer: stop song")

    def next_song(self):
        print("AudioPlayer: choice next song")

    def previous_song(self):
        print("AudioPlayer: choice previous song")

    def fast_forward(self, n):
        print(f"AudioPlayer: skip {n} second.")

    def rewind(self, n):
        print(f"AudioPlayer: rewind {n} second.")


class State:

    def __init__(self, player: AudioPlayer):
        self.player = player

    def click_lock(self):
        ...
    
    def click_play(self):
        ...

    def click_next(self, count=1):
        ...

    def click_previous(self, count=1):
        ...


class LockedState(State):

    def click_lock(self):
        print("on")
        self.player.state = ReadyState(self.player)


class ReadyState(State):

    def click_lock(self):
        print("off")
        self.player.state = LockedState(self.player)
    
    def click_play(self):
        self.player.start_playback()
        self.player.state = PlayingState(self.player)
    
    def click_next(self, count=1):
        self.player.next_song()
    
    def click_previous(self, count=1):
        self.player.previous_song()


class PlayingState(State):

    def click_lock(self):
        self.player.state = LockedState(self.player)
    
    def click_play(self):
        self.player.stop_playback()
        self.player.state = ReadyState(self.player)
    
    def click_next(self, count=1):
        if count >= 2:
            self.player.next_song()
        self.player.fast_forward(5)
    
    def click_previous(self, count=1):
        if count >= 2:
            self.player.previous_song()
        self.player.rewind(5)


player = AudioPlayer()

player.state.click_lock()  # lock -> ready
player.state.click_next()
player.state.click_play()  # ready -> playing
player.state.click_next(2)
player.state.click_play()
player.state.click_previous()
player.state.click_lock()  # playing -> lock

player.state.click_previous()  # ignored

on
AudioPlayer: choice next song
AudioPlayer: play song
AudioPlayer: choice next song
AudioPlayer: skip 5 second.
AudioPlayer: stop song
AudioPlayer: choice previous song
off
